# 📊 Análisis de Sensibilidad - Econova
## Plataforma Inteligente de Simulación Financiera

**Autor:** Diego (Responsable de Machine Learning)  
**Proyecto:** Econova - Simulación Financiera Empresarial con IA

### Objetivos:
1. Implementar **Simulación Monte Carlo** para análisis de riesgo
2. Identificar **variables clave** que más impactan los resultados financieros
3. Crear **escenarios** (optimista, pesimista, base) para decisiones
4. Calcular **intervalos de confianza** para predicciones

### Metodologías:
- **Monte Carlo**: Simulación estocástica con múltiples iteraciones
- **Análisis de Tornado**: Identificar variables más sensibles
- **Análisis What-If**: Escenarios de decisión

## 1. Importación de Librerías

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Para visualización (opcional)
try:
    import plotly.graph_objects as go
    import plotly.express as px
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False

np.random.seed(42)
print("✅ Librerías importadas correctamente")
print(f"📊 Plotly disponible: {PLOTLY_AVAILABLE}")

## 2. Simulación Monte Carlo para VAN

La simulación Monte Carlo permite evaluar la incertidumbre en el **Valor Actual Neto (VAN)** considerando que las variables de entrada (flujos de caja, tasa de descuento) no son valores fijos sino distribuciones de probabilidad.

In [ ]:
class SimulacionMonteCarlo:
    """
    Clase para realizar simulaciones Monte Carlo en análisis financiero.
    Permite evaluar la distribución de resultados posibles para VAN, TIR y otros indicadores.
    """
    
    def __init__(self, n_simulaciones: int = 10000, seed: int = 42):
        """
        Inicializa la simulación Monte Carlo.
        
        Args:
            n_simulaciones: Número de iteraciones de simulación
            seed: Semilla para reproducibilidad
        """
        self.n_simulaciones = n_simulaciones
        self.seed = seed
        np.random.seed(seed)
    
    def simular_van(
        self,
        inversion_inicial: float,
        flujos_base: List[float],
        tasa_descuento_base: float,
        variacion_flujos: float = 0.15,
        variacion_tasa: float = 0.02
    ) -> Dict[str, Any]:
        """
        Simula el VAN con variaciones en flujos de caja y tasa de descuento.
        
        Args:
            inversion_inicial: Inversión inicial del proyecto
            flujos_base: Flujos de caja esperados por periodo
            tasa_descuento_base: Tasa de descuento base
            variacion_flujos: Desviación estándar relativa de flujos (ej: 0.15 = 15%)
            variacion_tasa: Desviación estándar absoluta de la tasa
            
        Returns:
            Diccionario con resultados de la simulación
        """
        resultados_van = []
        n_periodos = len(flujos_base)
        
        for _ in range(self.n_simulaciones):
            # Simular flujos con distribución normal
            flujos_simulados = [
                max(0, np.random.normal(flujo, flujo * variacion_flujos))
                for flujo in flujos_base
            ]
            
            # Simular tasa de descuento
            tasa_simulada = max(0.01, np.random.normal(tasa_descuento_base, variacion_tasa))
            
            # Calcular VAN
            van = -inversion_inicial
            for t, flujo in enumerate(flujos_simulados, start=1):
                van += flujo / ((1 + tasa_simulada) ** t)
            
            resultados_van.append(van)
        
        resultados_van = np.array(resultados_van)
        
        # Estadísticas
        return {
            'van_medio': float(np.mean(resultados_van)),
            'van_mediana': float(np.median(resultados_van)),
            'desviacion_estandar': float(np.std(resultados_van)),
            'van_minimo': float(np.min(resultados_van)),
            'van_maximo': float(np.max(resultados_van)),
            'percentil_5': float(np.percentile(resultados_van, 5)),
            'percentil_95': float(np.percentile(resultados_van, 95)),
            'probabilidad_van_positivo': float(np.mean(resultados_van > 0)),
            'probabilidad_van_negativo': float(np.mean(resultados_van < 0)),
            'var_95': float(np.percentile(resultados_van, 5)),  # Value at Risk
            'cvar_95': float(np.mean(resultados_van[resultados_van <= np.percentile(resultados_van, 5)])),
            'n_simulaciones': self.n_simulaciones,
            'distribucion': resultados_van.tolist()[:1000]  # Muestra para gráficos
        }
    
    def simular_flujos_caja(
        self,
        flujos_base: List[float],
        variaciones: Dict[str, float] = None,
        tipo_distribucion: str = 'normal'
    ) -> Dict[str, Any]:
        """
        Simula flujos de caja con diferentes distribuciones de probabilidad.
        
        Args:
            flujos_base: Flujos de caja base
            variaciones: Dict con 'optimista', 'pesimista' como multiplicadores
            tipo_distribucion: 'normal', 'triangular', 'uniforme'
        """
        if variaciones is None:
            variaciones = {'pesimista': 0.7, 'optimista': 1.3}
        
        resultados = []
        
        for _ in range(self.n_simulaciones):
            flujos_sim = []
            for flujo in flujos_base:
                if tipo_distribucion == 'normal':
                    valor = np.random.normal(flujo, flujo * 0.15)
                elif tipo_distribucion == 'triangular':
                    valor = np.random.triangular(
                        flujo * variaciones['pesimista'],
                        flujo,
                        flujo * variaciones['optimista']
                    )
                else:  # uniforme
                    valor = np.random.uniform(
                        flujo * variaciones['pesimista'],
                        flujo * variaciones['optimista']
                    )
                flujos_sim.append(max(0, valor))
            resultados.append(flujos_sim)
        
        resultados = np.array(resultados)
        
        return {
            'flujos_medios': np.mean(resultados, axis=0).tolist(),
            'flujos_std': np.std(resultados, axis=0).tolist(),
            'percentil_5': np.percentile(resultados, 5, axis=0).tolist(),
            'percentil_95': np.percentile(resultados, 95, axis=0).tolist(),
            'total_medio': float(np.mean(np.sum(resultados, axis=1))),
            'total_std': float(np.std(np.sum(resultados, axis=1)))
        }

# Ejemplo de uso
print("✅ Clase SimulacionMonteCarlo definida")

In [ ]:
# Ejemplo: Simulación Monte Carlo para un proyecto de inversión
mc = SimulacionMonteCarlo(n_simulaciones=10000)

# Parámetros del proyecto
inversion = 100000  # S/ 100,000
flujos_esperados = [25000, 30000, 35000, 40000, 45000]  # 5 años
tasa_base = 0.12  # 12% de descuento

# Ejecutar simulación
resultado_mc = mc.simular_van(
    inversion_inicial=inversion,
    flujos_base=flujos_esperados,
    tasa_descuento_base=tasa_base,
    variacion_flujos=0.20,  # 20% de variación en flujos
    variacion_tasa=0.03     # 3% de variación en tasa
)

print("🎲 SIMULACIÓN MONTE CARLO - Análisis de VAN")
print("=" * 50)
print(f"📊 Inversión inicial: S/ {inversion:,.2f}")
print(f"📊 Flujos esperados: {flujos_esperados}")
print(f"📊 Tasa de descuento base: {tasa_base:.1%}")
print(f"📊 Simulaciones realizadas: {resultado_mc['n_simulaciones']:,}")
print()
print("📈 RESULTADOS:")
print(f"   VAN Medio: S/ {resultado_mc['van_medio']:,.2f}")
print(f"   VAN Mediana: S/ {resultado_mc['van_mediana']:,.2f}")
print(f"   Desviación Estándar: S/ {resultado_mc['desviacion_estandar']:,.2f}")
print(f"   VAN Mínimo: S/ {resultado_mc['van_minimo']:,.2f}")
print(f"   VAN Máximo: S/ {resultado_mc['van_maximo']:,.2f}")
print()
print("📊 INTERVALOS DE CONFIANZA (90%):")
print(f"   Percentil 5%: S/ {resultado_mc['percentil_5']:,.2f}")
print(f"   Percentil 95%: S/ {resultado_mc['percentil_95']:,.2f}")
print()
print("⚠️ ANÁLISIS DE RIESGO:")
print(f"   Probabilidad VAN > 0: {resultado_mc['probabilidad_van_positivo']:.1%}")
print(f"   Probabilidad VAN < 0: {resultado_mc['probabilidad_van_negativo']:.1%}")
print(f"   VaR (95%): S/ {resultado_mc['var_95']:,.2f}")
print(f"   CVaR (95%): S/ {resultado_mc['cvar_95']:,.2f}")

## 3. Análisis de Sensibilidad de Variables

El análisis de sensibilidad determina cuánto cambia el resultado (VAN, TIR) cuando variamos una variable manteniendo las demás constantes. Esto ayuda a identificar las **variables críticas** del proyecto.

In [ ]:
class AnalisisSensibilidad:
    """
    Clase para realizar análisis de sensibilidad en proyectos financieros.
    Permite identificar variables críticas y su impacto en los resultados.
    """
    
    @staticmethod
    def calcular_van(inversion: float, flujos: List[float], tasa: float) -> float:
        """Calcula el VAN de un proyecto."""
        van = -inversion
        for t, flujo in enumerate(flujos, start=1):
            van += flujo / ((1 + tasa) ** t)
        return van
    
    def analisis_univariado(
        self,
        inversion_inicial: float,
        flujos_base: List[float],
        tasa_base: float,
        variable: str,
        rango_variacion: Tuple[float, float] = (-0.3, 0.3),
        n_pasos: int = 20
    ) -> Dict[str, Any]:
        """
        Analiza cómo varía el VAN al cambiar una sola variable.
        
        Args:
            inversion_inicial: Inversión inicial
            flujos_base: Flujos de caja base
            tasa_base: Tasa de descuento base
            variable: 'inversion', 'flujos', 'tasa'
            rango_variacion: Rango de variación como porcentaje
            n_pasos: Número de puntos a evaluar
        """
        variaciones = np.linspace(rango_variacion[0], rango_variacion[1], n_pasos)
        resultados = []
        
        van_base = self.calcular_van(inversion_inicial, flujos_base, tasa_base)
        
        for var in variaciones:
            if variable == 'inversion':
                inv_mod = inversion_inicial * (1 + var)
                van = self.calcular_van(inv_mod, flujos_base, tasa_base)
            elif variable == 'flujos':
                flujos_mod = [f * (1 + var) for f in flujos_base]
                van = self.calcular_van(inversion_inicial, flujos_mod, tasa_base)
            elif variable == 'tasa':
                tasa_mod = tasa_base * (1 + var)
                van = self.calcular_van(inversion_inicial, flujos_base, tasa_mod)
            else:
                raise ValueError(f"Variable no reconocida: {variable}")
            
            resultados.append({
                'variacion_pct': var * 100,
                'van': van,
                'cambio_van': van - van_base,
                'cambio_van_pct': ((van - van_base) / abs(van_base)) * 100 if van_base != 0 else 0
            })
        
        df_resultados = pd.DataFrame(resultados)
        
        # Calcular elasticidad (sensibilidad)
        elasticidad = np.polyfit(df_resultados['variacion_pct'], df_resultados['cambio_van_pct'], 1)[0]
        
        return {
            'variable': variable,
            'van_base': van_base,
            'resultados': df_resultados.to_dict('records'),
            'elasticidad': elasticidad,
            'interpretacion': f"Por cada 1% de cambio en {variable}, el VAN cambia {elasticidad:.2f}%"
        }
    
    def analisis_tornado(
        self,
        inversion_inicial: float,
        flujos_base: List[float],
        tasa_base: float,
        variacion: float = 0.20
    ) -> Dict[str, Any]:
        """
        Análisis de tornado para comparar sensibilidad de múltiples variables.
        
        Args:
            variacion: Variación porcentual a aplicar (ej: 0.20 = ±20%)
        """
        van_base = self.calcular_van(inversion_inicial, flujos_base, tasa_base)
        
        resultados_tornado = []
        
        # Análisis para inversión
        van_inv_bajo = self.calcular_van(inversion_inicial * (1 - variacion), flujos_base, tasa_base)
        van_inv_alto = self.calcular_van(inversion_inicial * (1 + variacion), flujos_base, tasa_base)
        resultados_tornado.append({
            'variable': 'Inversión Inicial',
            'van_bajo': van_inv_alto,  # Invertido: mayor inversión = menor VAN
            'van_alto': van_inv_bajo,
            'rango': abs(van_inv_alto - van_inv_bajo)
        })
        
        # Análisis para flujos
        van_flujos_bajo = self.calcular_van(inversion_inicial, [f * (1 - variacion) for f in flujos_base], tasa_base)
        van_flujos_alto = self.calcular_van(inversion_inicial, [f * (1 + variacion) for f in flujos_base], tasa_base)
        resultados_tornado.append({
            'variable': 'Flujos de Caja',
            'van_bajo': van_flujos_bajo,
            'van_alto': van_flujos_alto,
            'rango': abs(van_flujos_alto - van_flujos_bajo)
        })
        
        # Análisis para tasa
        van_tasa_bajo = self.calcular_van(inversion_inicial, flujos_base, tasa_base * (1 - variacion))
        van_tasa_alto = self.calcular_van(inversion_inicial, flujos_base, tasa_base * (1 + variacion))
        resultados_tornado.append({
            'variable': 'Tasa de Descuento',
            'van_bajo': van_tasa_alto,  # Mayor tasa = menor VAN
            'van_alto': van_tasa_bajo,
            'rango': abs(van_tasa_alto - van_tasa_bajo)
        })
        
        # Ordenar por impacto
        resultados_tornado.sort(key=lambda x: x['rango'], reverse=True)
        
        return {
            'van_base': van_base,
            'variacion_aplicada': variacion,
            'resultados': resultados_tornado,
            'variable_mas_sensible': resultados_tornado[0]['variable']
        }
    
    def escenarios(
        self,
        inversion_inicial: float,
        flujos_base: List[float],
        tasa_base: float,
        escenarios_config: Dict[str, Dict] = None
    ) -> Dict[str, Any]:
        """
        Análisis de escenarios: pesimista, base, optimista.
        """
        if escenarios_config is None:
            escenarios_config = {
                'pesimista': {'flujos_mult': 0.75, 'tasa_mult': 1.25},
                'base': {'flujos_mult': 1.0, 'tasa_mult': 1.0},
                'optimista': {'flujos_mult': 1.25, 'tasa_mult': 0.85}
            }
        
        resultados = {}
        
        for nombre, config in escenarios_config.items():
            flujos_esc = [f * config['flujos_mult'] for f in flujos_base]
            tasa_esc = tasa_base * config['tasa_mult']
            van = self.calcular_van(inversion_inicial, flujos_esc, tasa_esc)
            
            resultados[nombre] = {
                'van': van,
                'flujos': flujos_esc,
                'tasa': tasa_esc,
                'decision': 'ACEPTAR' if van > 0 else 'RECHAZAR'
            }
        
        return {
            'escenarios': resultados,
            'rango_van': resultados['optimista']['van'] - resultados['pesimista']['van'],
            'recomendacion': self._generar_recomendacion(resultados)
        }
    
    def _generar_recomendacion(self, escenarios: Dict) -> str:
        """Genera recomendación basada en escenarios."""
        van_pesimista = escenarios['pesimista']['van']
        van_base = escenarios['base']['van']
        van_optimista = escenarios['optimista']['van']
        
        if van_pesimista > 0:
            return "PROYECTO ROBUSTO: Rentable incluso en escenario pesimista"
        elif van_base > 0 and van_pesimista < 0:
            return "PROYECTO CON RIESGO MODERADO: Rentable en escenario base pero no en pesimista"
        elif van_optimista > 0 and van_base < 0:
            return "PROYECTO ARRIESGADO: Solo rentable en escenario optimista"
        else:
            return "PROYECTO NO RECOMENDADO: No rentable en ningún escenario"

print("✅ Clase AnalisisSensibilidad definida")

In [ ]:
# Ejemplo: Análisis de Tornado
sensibilidad = AnalisisSensibilidad()

resultado_tornado = sensibilidad.analisis_tornado(
    inversion_inicial=100000,
    flujos_base=[25000, 30000, 35000, 40000, 45000],
    tasa_base=0.12,
    variacion=0.20
)

print("🌪️ ANÁLISIS DE TORNADO")
print("=" * 60)
print(f"VAN Base: S/ {resultado_tornado['van_base']:,.2f}")
print(f"Variación aplicada: ±{resultado_tornado['variacion_aplicada']:.0%}")
print()
print("📊 Impacto por Variable (ordenado de mayor a menor):")
print("-" * 60)
for res in resultado_tornado['resultados']:
    print(f"   {res['variable']:20} | Rango: S/ {res['rango']:>12,.2f}")
    print(f"   {'':20} | Bajo: S/ {res['van_bajo']:>10,.2f} | Alto: S/ {res['van_alto']:>10,.2f}")
    print()

print(f"⚠️ Variable más sensible: {resultado_tornado['variable_mas_sensible']}")

In [ ]:
# Ejemplo: Análisis de Escenarios
resultado_escenarios = sensibilidad.escenarios(
    inversion_inicial=100000,
    flujos_base=[25000, 30000, 35000, 40000, 45000],
    tasa_base=0.12
)

print("📊 ANÁLISIS DE ESCENARIOS")
print("=" * 60)
for nombre, datos in resultado_escenarios['escenarios'].items():
    emoji = "😰" if nombre == "pesimista" else "😊" if nombre == "optimista" else "😐"
    print(f"\n{emoji} Escenario {nombre.upper()}:")
    print(f"   VAN: S/ {datos['van']:,.2f}")
    print(f"   Tasa aplicada: {datos['tasa']:.1%}")
    print(f"   Decisión: {datos['decision']}")

print(f"\n📏 Rango de VAN: S/ {resultado_escenarios['rango_van']:,.2f}")
print(f"\n💡 {resultado_escenarios['recomendacion']}")

## 4. Resumen y Exportación de Funciones

Las clases `SimulacionMonteCarlo` y `AnalisisSensibilidad` serán integradas en el servicio ML del backend para uso vía API.